# Listenverbindungen Public

In [ ]:
# import libraries

import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from datetime import datetime
# import re
# import os
# import json
# import requests

In [3]:
# import data



---

In [ ]:
# start cleaning and analysis

